In [1]:
import requests
import os
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO

# Get all links from Catalog page:

In [2]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}

In [3]:
# Define the URL you want to scrape
url = 'https://www.gucci.com/it/it/ca/women/ready-to-wear-for-women-c-women-readytowear'

In [4]:
# Send an HTTP request to the URL
response = requests.get(url, headers = header)
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
item_list = soup.select('article a')

In [6]:
for item in item_list: 
    link = 'https://www.gucci.com' + item['href']
    print(link)

https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/knitwear-for-women/knitted-dresses-for-women/pleated-dress-with-web-trim-p-786401XKD2R9288
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/knitwear-for-women/knitted-dresses-for-women/polo-dress-with-web-trim-p-788864XKD269791
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/outerwear-for-women/gg-jacket-with-web-p-801106Z40399692
https://www.gucci.com/it/it/pr/whats-new/new-in/this-week-women/ready-to-wear-for-women/short-pleated-skirt-with-striped-trim-p-691712XKD279288
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/activewear-for-women/technical-jersey-short-with-web-p-785500XJGK69039
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/activewear-for-women/short-technical-jersey-dress-with-web-p-785490XJGK69039
https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/activewear-for-women/jersey-drill-jacket-with-web-p-785483XJGK39799
https://www.gucci.com/it/it/pr/women/r

# Get from specific product page:
- Relevant pictures
- item description

In [7]:
# New Link
link = 'https://www.gucci.com/it/it/pr/women/ready-to-wear-for-women/skirts-for-women/midi-long-skirts-for-women/gg-supreme-oxford-cotton-skirt-p-788904ZAM9M5307'

### look for description

In [14]:
# Send an HTTP request to the URL
response_2 = requests.get(link, headers = header)
soup_2 = BeautifulSoup(response_2.content, 'html.parser')

In [17]:
list_2 = soup_2.find_all(attrs={"class": "product-detail"} )

In [66]:
list_2[0].p.text.replace('\n', '').replace('\t', '')

'Ispirato allo spirito estivo e ai beach club della costa italiana, questo articolo fa parte della linea Gucci Lido. Questa gonna svasata è realizzata in cotone Oxford GG Supreme rosa chiaro e impreziosita da un nastro Web blu e rosso sulla vita.'

In [58]:
description = ''

for a in list_2[0].ul.find_all('li'):
    description = description + ' ' + a.text
    
description

" Cotone Oxford GG Supreme rosa chiaro Finiture con nastro Web rosso e blu Senza fodera Fascia in vita aderente Taglio svasato Modello a pieghe Chiusura con cerniera nascosta Lunghezza: 73 cm; taglia 40 (IT) Fabbricato in Italia Il prodotto raffigurato nell'immagine è taglia 40 (IT)"

In [67]:
list_2[0].p.text.replace('\n', '').replace('\t', '') + ' ' + description

"Ispirato allo spirito estivo e ai beach club della costa italiana, questo articolo fa parte della linea Gucci Lido. Questa gonna svasata è realizzata in cotone Oxford GG Supreme rosa chiaro e impreziosita da un nastro Web blu e rosso sulla vita.  Cotone Oxford GG Supreme rosa chiaro Finiture con nastro Web rosso e blu Senza fodera Fascia in vita aderente Taglio svasato Modello a pieghe Chiusura con cerniera nascosta Lunghezza: 73 cm; taglia 40 (IT) Fabbricato in Italia Il prodotto raffigurato nell'immagine è taglia 40 (IT)"

In [ ]:
# Log results in excel:

def create_description(soup_2):
    list_2 = soup_2.find_all(attrs={"class": "product-detail"} )
    list_2[0].p.text.replace('\n', '').replace('\t', '')

    description = ''

    for a in list_2[0].ul.find_all('li'):
        description = description + ' ' + a.text
        
    

    return list_2[0].p.text.replace('\n', '').replace('\t', '') + ' ' + description
 

### look for image

In [8]:
# Send an HTTP request to the URL
response_1 = requests.get(link, headers = header)
soup_1 = BeautifulSoup(response_1.content, 'html.parser')

In [9]:
list_1 = soup_1.find_all('picture' )


In [10]:
list_1 = soup_1.find_all(attrs={"data-image-size": "small-retina"})

In [11]:
keeping = ['001_100','004_100','005_100']
# 001: Prod
# 004: back
# 005: 3/4 image


In [12]:
#keeping = ['001', '002', '004', '005', '012']

def contains_required_number(line):
    return any(num in line for num in keeping)

In [13]:
output_dir = "Img"
os.makedirs(output_dir, exist_ok=True)

In [13]:
i = 0
for item in list_1:
    i+=1 
    if (i == len(list_1)) | (i == 1):
        continue
    elif not(contains_required_number(item['srcset'])) :
        continue
    else:
        print(item['srcset'] ) 
        url = "https:" + item['srcset']

        # Download the image
        response = requests.get(url, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))
        # Save the image
        img_name = os.path.join(output_dir, f"Gucci5_{i}.jpg") 
        img_resized.save(img_name)
        print(f"Saved {img_name}")

//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805269/788904_ZAM9M_5307_001_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_2.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805271/788904_ZAM9M_5307_004_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_5.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805272/788904_ZAM9M_5307_005_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_6.jpg
//media.gucci.com/style/DarkGray_Center_0_0_980x980/1713805274/788904_ZAM9M_5307_012_100_0000_Light-Gonna-in-cotone-Oxford-GG-Supreme.jpg
Saved Img\Gucci5_8.jpg
